In [1]:
import pandas as pd
import json
import numpy as np
import spacy
import nltk
from nltk.corpus import stopwords
import re
import gensim
import math

C:\Users\karan\Miniconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
global AVG_TAGS_PER_VIDEO, US_CA_GB_TOKEN_CORPUS

In [3]:
#get rid of the punctuations and set all characters to lowercase
RE_PREPROCESS = r'\W+|\d+' #the regular expressions that matches all non-characters

#get rid of punctuation and make everything lowercase
#the code belows works by looping through the array of text
#for a given piece of text we invoke the `re.sub` command where we pass in the regular expression, a space ' ' to
#subsitute all the matching characters with
#we then invoke the `lower()` method on the output of the re.sub command
#to make all the remaining characters
#the cleaned document is then stored in a list
#once this list has been filed it is then stored in a numpy array

In [66]:
RE_REMOVE_URLS = r'http\S+'

In [67]:
def processFeatures(desc):
    try:
        desc = re.sub(RE_REMOVE_URLS, ' ', desc)
        return re.sub(RE_PREPROCESS, ' ', desc)
    except:
        return " "

In [68]:
def processDataFrame(data_frame, country_code='US'):
    data_frame.sort_values(by=['video_id', 'trending_date'], ascending=True, inplace=True)
    grouped_videos = data_frame.groupby(['video_id']).last().reset_index()
    
    #Reading categories from the json file depending on country_code
    json_location = './data/' + country_code +'_category_id.json'
    with open(json_location) as data_file:
        data = json.load(data_file)    
    categories = []
    for item in data['items']:
        category = {}
        category['category_id'] = int(item['id'])
        category['title'] = item['snippet']['title']
        categories.append(category)

    categories_df = pd.DataFrame(categories)
    # Merging videos data with category data
    final_df = grouped_videos.merge(categories_df, on = ['category_id'])
    final_df.rename(columns={'title_y': 'category', 'title_x': 'video_name'}, inplace=True)
    
    # Creating a features column that consists all features used for prediction.
    final_df['video_features'] = ''
    if final_df['tags'].astype(str) is not None:
        final_df['video_features'] += final_df['tags'].astype(str)
    
    if final_df['video_name'].astype(str) is not None:
        final_df['video_features'] += final_df['video_name'].astype(str)

    if final_df['channel_title'].astype(str) is not None:
        final_df['video_features'] += final_df['channel_title'].astype(str)
        
    if final_df['description'].astype(str) is not None:
        final_df['video_features'] += final_df['description'].astype(str)
    
    #final_df['video_features'] = final_df['tags'].astype(str) + final_df['video_name'].astype(str) \
    #                    + final_df['channel_title'].astype(str) + final_df['description'] + final_df['category']
        
    final_df['video_features'] = final_df['video_features'].apply(processFeatures)
    final_df['video_features'] = final_df['video_features'].apply(processFeatures)
    return final_df

In [69]:
def removeStopwords(documents):
    stopwords_list = stopwords.words('english')
    processed_corpus = []
    for document in documents:
        processed_document = []
        for word in document.split():
            if word not in stopwords_list:
                processed_document.append(word)
        processed_corpus.append(processed_document)
    return processed_corpus

In [70]:
def processCorpus(feature_corpus):
    feature_corpus = [comment.lower() for comment in feature_corpus]
    processed_feature_corpus = removeStopwords(feature_corpus)
    return processed_feature_corpus

In [71]:
def trainModel(token_corpus):
    model = gensim.models.Word2Vec(sentences=token_corpus, min_count=1, size = 32)
    model.train(token_corpus, total_examples=model.corpus_count, epochs=model.iter)
    model.save('word2vec_model.w2v')
    return model

In [72]:
def recommendTags(word2vec_model, input_words = ['trump', 'president'], number_of_tags = 10, model_name = 'word2vec_model.w2v'):
    tags = []
    if word2vec_model is None:
        gensim.models.Word2Vec.load(model_name)
    try:
        tags = word2vec_model.most_similar(positive=input_words, topn=number_of_tags)
    except:
        US_CA_GB_TOKEN_CORPUS.append(input_words)
        word2vec_model.build_vocab(US_CA_GB_TOKEN_CORPUS, update=True)
        word2vec_model.train(US_CA_GB_TOKEN_CORPUS, total_examples=word2vec_model.corpus_count, epochs=word2vec_model.iter)
        word2vec_model.save(model_name)
        tags = word2vec_model.most_similar(positive=input_words, topn=number_of_tags)
    
    return tags

Running the algorithm for US, CA, and GB videos

In [79]:
us_videos_df = pd.read_csv('./data/USvideos.csv')
us_final_df = processDataFrame(us_videos_df, country_code='US')

In [80]:
ca_videos_df = pd.read_csv('./data/CAvideos.csv')
ca_final_df = processDataFrame(ca_videos_df, country_code='CA')

In [81]:
gb_videos_df = pd.read_csv('./data/GBvideos.csv')
gb_final_df = processDataFrame(gb_videos_df, country_code='GB')

In [82]:
us_ca_gb_final_df = pd.concat([us_final_df, ca_final_df, gb_final_df])

In [83]:
us_ca_gb_final_df.reset_index(inplace=True)

In [84]:
US_CA_GB_TOKEN_CORPUS = processCorpus(us_ca_gb_final_df['video_features'].values)

In [85]:
total_tags = 0
for tag_list in us_ca_gb_final_df['tags'].values:
    total_tags += len(tag_list.split('|'))
print('Total tags in the corpus:', total_tags)
AVG_TAGS_PER_VIDEO = math.ceil(total_tags/len(us_ca_gb_final_df))
print('Average number of tags per video:', AVG_TAGS_PER_VIDEO)

Total tags in the corpus: 56012
Average number of tags per video: 19


In [86]:
us_ca_gb_final_df['video_features'] = US_CA_GB_TOKEN_CORPUS

In [87]:
len(US_CA_GB_TOKEN_CORPUS)

3029

In [88]:
word2vec_model = trainModel(US_CA_GB_TOKEN_CORPUS)

Computing average number of tags per video

In [89]:
tags = recommendTags(word2vec_model, input_words=['trump', 'president'], 
                     number_of_tags=AVG_TAGS_PER_VIDEO, 
                     model_name = 'word2vec_model.w2v')

In [90]:
tags

[('donald', 0.9859254360198975),
 ('pocahontas', 0.9739551544189453),
 ('hillary', 0.9519702792167664),
 ('clinton', 0.9507759809494019),
 ('napahanga', 0.9467853307723999),
 ('manafort', 0.9466699361801147),
 ('sacked', 0.9448211193084717),
 ('cozying', 0.94069504737854),
 ('isp', 0.939950168132782),
 ('turkeywashington', 0.9399107098579407),
 ('russia', 0.9380573034286499),
 ('colluding', 0.9364501237869263),
 ('probe', 0.9335001707077026),
 ('hostility', 0.9334907531738281),
 ('denies', 0.932379424571991),
 ('warren', 0.9309561252593994),
 ('ungrateful', 0.9308549165725708),
 ('rodrigo', 0.9304343461990356),
 ('kellyanne', 0.9302242994308472)]

## Dividing the dataset into training (80%) and testing sets (20%).

In [91]:
np.random.seed(seed=13579)
us_ca_gb_final_df_shuffled = us_ca_gb_final_df.iloc[np.random.permutation(len(us_ca_gb_final_df))]

In [92]:
train_size = 0.80
us_ca_gb_df_train = us_ca_gb_final_df_shuffled[:int((train_size)*len(us_ca_gb_final_df_shuffled))]
us_ca_gb_df_test = us_ca_gb_final_df_shuffled[int((train_size)*len(us_ca_gb_final_df_shuffled)):]

In [93]:
w2v_train_model = gensim.models.Word2Vec(sentences=us_ca_gb_df_train['video_features'], min_count=1, size = 32)
w2v_train_model.train(us_ca_gb_df_train['video_features'].values, total_examples=w2v_train_model.corpus_count, epochs=w2v_train_model.iter)
w2v_train_model.save('w2v_train_model.w2v')

In [94]:
w2v_train_model = gensim.models.Word2Vec.load('w2v_train_model.w2v')

In [95]:
predicted_tags = []
for idx in us_ca_gb_df_test.index:
    video_features = us_ca_gb_df_test.loc[idx, 'video_features']
    if len(video_features) > 0:
        tag_probability_list = recommendTags(w2v_train_model, input_words=video_features,
                                             number_of_tags=AVG_TAGS_PER_VIDEO, 
                                             model_name = 'w2v_train_model.w2v')
        predicted_tags.append([tag[0] for tag in tag_probability_list])

    

In [96]:
us_ca_gb_df_test['predicted_tags'] = predicted_tags

C:\Users\karan\Miniconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [97]:
us_ca_gb_df_test['tags'] = us_ca_gb_df_test['tags'].apply(processFeatures)

C:\Users\karan\Miniconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [98]:
match_found = 0
count = 0
for idx in us_ca_gb_df_test.index:
    tag_list = us_ca_gb_df_test.loc[idx,'tags'].lower()
    tag_list = tag_list.split(' ')
    predicted_tag_list = us_ca_gb_df_test.loc[idx, 'predicted_tags']
    
    print(tag_list)
    print(predicted_tag_list)
    print('=============================')
    for i in range(len(tag_list)):
        if tag_list[i] in predicted_tag_list:
            match_found += 1
            break
    count += 1
print('Match found: ', match_found )
print('Accuracy: ', match_found/len(us_ca_gb_df_test))
    
    


['battle', 'universe', 'nerf', 'battle', 'universe', 'nerf', 'challenge', 'epic', 'nerf', 'battle', 'nerf', 'gun', 'fight', 'nerf', 'fight', 'team', 'edge', 'hi', '', 'hi', '', 'nerf', 'hi', '', 'studios', 'matthias', 'vlogs', 'nerf', 'nurf', 'nerf', 'war', 'nerf', 'battle', 'nerf', 'dungeons', 'dragons', 'team', 'edge', 'nerf', 'nerf', 'war', 'nerf', 'attack', '']
['sub', 'negative', 'opening', 'wife', 'shooter', 'fire', 'working', 'ngame', 'sense', 'stop', 'trivia', 'pokemon', 'met', 'difficulties', 'mestalla', 'critically', 'already', 'begin', 'diego']
['graham', 'norton', 'graham', 'norton', 'show', 'official', 'entertainment', 'chat', 'show', 'graham', 'norton', 'full', 'episodes', 'the', 'graham', 'norton', 'show', 'full', 'eps', 'graham', 'norton', 'full', 'eps', 'watch', 'graham', 'norton', 'online', 'jason', 'momoa', 'hugh', 'grant', 'sarah', 'millican', 'kelly', 'clarkson', 'graham', 'norton', 'red', 'chair', 'game', 'of', 'thrones', '']
['showit', 'bee', 'reilly', 'roba', 's

['monaco', 'vs', 'psg', '', '', '', '', 'highlights', 'goals', '', '', 'november', '', '', 'football', 'minds', '', 'x', '', '']
['numbers', 'figure', 'valverde', 'flight', 'mahrez', 'encounter', 'werner', 'nrepeat', 'turning', 'install', 'dairies', 'guaranteed', 'firewood', 'quid', 'placed', 'nroyal', 'equaliser', 'phase', 'opponents']
['lion', 'king', 'the', 'lion', 'king', 'lion', 'lion', 'simba', 'simba', 'nala', 'timon', 'pumba', 'scar', 'mufasa', 'disney', 'disney', 'animation', 'animation', 'cartoons', 'comedy', 'cinematography', 'film', 'studies', 'earthling', 'cinema', 'garyx', 'wormuloid', 'thugnotes', 'wisecrack', 'edition', 'wisecrack', '', '', 'bit', 'philosophy', 'what', 'went', 'wrong', 'quick', 'take', '']
['occurred', 'perspective', '高鐵一地兩檢通過', 'خلف', 'n포항', 'passengers', 'enforced', 'njinkx', 'jinger', 'resurrective', 'bueno', 'knoos', 'n미국', 'бизнес', 'hershesons', '한국인', '로', 'ahab', '不要等到刀架在脖子上才知道疼']
['iphone', 'x', 'iphonex', 'makeup', 'iphonex', 'test', 'new', 'i

['sri', 'lanka', 'sinhala', 'tv', 'derana', 'derana', 'dtv', 'lanka', 'premium', 'entertainment', 'deweni', 'inima', 'dewana', 'inima', 'deveni', 'inima', 'devana', 'inima', 'cricket', 'sri', 'lanka', 'cricket', 'cricket', 'match', 'dawana', 'inima', 'deweni', 'enima', 'deveni', 'enima', 'deweni', '', 'nd', 'innings', 'innings', 'second', 'inning', 'cricket', 'team', 'saranga', 'mendis', 'janaka', 'siriwardana', 'sidu', 'saman', 'edirimuni', 'best', 'teledrama', 'sri', 'lankan', 'teledrama', 'top', 'teledrama', 'sri', 'lanka', 's', 'best', 'play', 'cricket', 'love', 'romantic', 'school', 'school', 'love', 'adventure', 'fight', '']
['portrayal', 'ሰንሰለት', 'saba', 'zemen', 'baaghi', 'ᴴᴰ', 'sarmad', 'hiyab', 'hearted', 'ምዕራፍ', 'khamoshi', 'officialgustakh', 'kazmibest', 'senselet', 'japantokyo', 'dramasbaaghi', 'bitto', 'ክፍል', 'ka']
['late', 'night', 'seth', 'meyers', 'closer', 'look', 'roy', 'moore', 'trump', 'putin', 'nbc', 'nbc', 'tv', 'television', 'funny', 'talk', 'show', 'comedy', 'h

['dumating', 'charming', 'urges', 'vladimir', 'nobyembre', 'maynila', 'held', 'ottawa', 'gala', 'tramp', 'ncnn', 'hearings', 'pilipinas', 'aquino', 'calling', 'putin', 'pangulo', 'khou', 'addressed']
['zoe', 'sugg', 'zoella', 'zoe', 'vlog', 'vlogging', 'vlogs', 'daily', 'every', 'day', 'british', 'in', 'the', 'life', 'brighton', 'nala', 'puppy', 'pug', 'alfie', 'deyes', 'pointlessblog', 'beauty', 'fashion', 'makeup', 'lifestyle', 'life', 'clothes', 'outfits', 'dog', 'home', 'interiors', 'books', 'food', 'baking', '']
['magical', 'spirit', 'pointlessblogvlogs', 'bar', 'pieces', 'girlfriends', 'blogger', 'clumsy', 'mecca', 'surprising', 'ndie', 'ivy', 'dress', 'loving', 'nutrition', 'nikki', 'encounters', 'yelling', 'designed']
['jamie', 'oliver', 'meatballs', 'and', 'pasta', 'the', 'best', 'italian', 'spaghetti', 'and', 'meatballs', 'spaghetti', 'and', 'meatballs', 'recipe', 'spaghetti', 'meatballs', 'gennaro', 'contaldo', 'quick', 'and', 'easy', 'spaghetti', 'and', 'meatballs', 'quick'

['essiebutton', 'estée', 'lalonde', 'estee', 'lalonde', 'essie', 'button', 'essie', 'no', 'makeup', 'makeup', 'drugstore', 'makeup', 'tutorial', 'fashion', 'cute', 'outfits', 'makeup', 'tutorials', 'healthy', 'foods', 'eye', 'makeup', 'outfit', 'ideas', 'natural', 'makeup', 'easy', 'healthy', 'recipes', 'skin', 'care', 'products', 'best', 'makeup', 'beauty', 'favourites', 'favorites', 'november', '', '', 'haul', 'new', 'in', 'glossier', 'you', 'perfume', 'review', 'ouai', 'messy', 'hair', 'charlotte', 'tilbury', 'skincare', 'lancome', 'highlighter', 'glow', 'scandinavian', 'rupi', 'kaur', 'fashion', '']
['kits', 'tati', 'nmacarons', 'narmani', 'ncover', 'string', 'lovecraft', 'blender', 'nblue', 'nmake', 'n컨투어', 'amelia', 'airflash', 'ndior', 'nfinal', 'ntravel', 'npixi', 'nvest', 'color']
['', 'none', '']
['groff', 'smashbits', 'nlinks', 'npick', 'nixxiom', 'sensation', 'silent', 'ntrack', 'gabe', 'nic', 'alum', 'raja', 'cork', 'tee', 'akademiks', 'tchaikovsky', 'mailing', 'nregan', '

...

['vamsam', 'mehta', 'baaghi', 'ᴴᴰ', 'taarak', 'urdu', 'dramasbaaghi', 'aur', 'geoshayad', 'officialgustakh', 'chronicles', 'geokhaani', 'galvanax', 'alif', 'yesterday', 'gumraah', 'allah', 'risesneo', 'pal']
['吳宗憲', '吳姍儒', '小明星大跟班', 'sandy', 'jacky', 'wu', '憲哥', '中天綜合台', 'ch', '', '中天電視台', '我愛小明星大跟班', '舒子晨', '康茵茵', '夏語心', '詹子晴', '成語蕎', '熊熊', '嘻小瓜', '斯亞', '小賴', '友情', '友誼', '舞蹈表演', '']
['워터프루프', '이렇게', 'n國家與國際事務專家', 'bilalis', 'nthimios', '펜슬', 'n前立法委員', '갈라져', '콜', 'nпоединок', '같이', '所有最新最热门的华语视频', '나왔습니다', '안내', '하우스', 'shehzad', 'shaharyar', 'naitor', '尽在看看新']
['vox', 'com', 'vox', 'explain', 'race', 'poverty', 'tickets', 'investigation', 'propublica', 'pedestrian', 'infrastructure', 'walking', 'walking', 'while', 'black', 'profiling', 'racial', 'profiling', 'policing', 'police', 'jacksonville', 'florida', 'duval', 'county', 'citation', 'law', 'enforcement', 'urban', 'planning', 'topher', 'sanders', 'ben', 'conarck', 'jaywalking', 'devonte', 'shipman', 'jso', 'florida', 'times', 'uni

['сериал', 'сериал', '', '', 'сериал', '', 'к', 'сериал', 'слуга', 'народа', '', '', 'слуга', 'народа', '', '', 'слуга', 'народа', '', '', 'сезон', 'слуга', 'народа', 'сериал', 'слуга', 'народа', '', '', 'серия', 'слуга', 'народа', 'новый', 'сезон', 'новый', 'сезон', 'слуга', 'народа', 'слуга', 'народа', 'премьера', 'слуга', 'народа', 'смотреть', 'онлайн', 'сериал', 'слуга', 'народа', 'онлайн', 'смотреть', 'новый', 'слуга', 'народа', 'слуга', 'народа', 'sluga', 'narodu', 'слуга', 'народа', 'квартал', '', '', 'смотреть', 'слуга', 'народа', 'когда', 'выйдет', 'премьера', '', 'к', 'премьера', '', '', 'президент', 'украины', '']
['را', 'در', '市議會', '台南', '锦绣缘华丽冒险noah', 'binetna', 'جد', '柱柱姐', '蔡正元', 'آمده', 'برنامه', '真道理性', 'nhar', 'پشت', 'ਹਨ', '黃敬平', 'фестиваль', '董智森', '三个哥哥对老四疼爱有加']
['mayo', 'clinic', 'health', 'care', 'issue', 'healthcare', 'science', 'field', 'of', 'study', 'organ', 'transplant', 'essam', 'and', 'dalal', 'obaid', 'center', 'for', 'reconstructive', 'transplant', 'surg

['kpop', '', 'thek', '원더케이', 'loen', '로엔', '뮤비', '티져', 'mv', 'teaser', '신곡', 'new', 'song', '한류', 'hallyu', 'ロエン', 'ミュージック', 'ミュージックビデオ', 'ケーポップ', '韓国の歌', 'アイドル', '韓流', '韓国', '아이돌', 'idol', 'kard', 'you', 'in', 'me', '비엠', '제이셉', '전소민', '전지우', 'bm', 'jseph', 'somin', 'jiwoo', '']
['nnaveen', 'nsebastian', 'suju', '펑키하고', 'n포항', '国家治理现代化', '不要等到刀架在脖子上才知道疼', 'nrecorded', 'kamal', '동선변화가', '주소지', 'خلف', 'n但是', '习近平倡导', '목포', 'quizon', '미', 'ولوجيستيك', 'бизнес']
['', 'none', '']
['objective', 'fledged', 'generator', 'resistance', 'derwent', 'phase', 'breach', 'tvmsg', 'warplane', 'threatening', 'nroyal', 'chopper', 'showcould', 'إخبارية', 'fnaf', 'saves', 'وزير', 'heroic', 'flight']
['marvel', 'comics', 'hip', 'hop', 'black', 'eyed', 'peas', 'masters', 'of', 'the', 'sun', 'the', 'zombie', 'chronicles', 'augmented', 'reality', 'technology', 'graphic', 'novels', 'trailer', 'mobile', 'app', 'music', 'social', 'justice', 'common', 'mary', 'j', 'blidge', 'krs', 'one', 'snoop', 'snoop', 'dogg',

['fachélama', 'avons', 'vloggeur', 'nles', 'carmanía', 'titres', 'nom', 'élevé', 'taupe', 'nmot', 'moffet', 'superbes', 'vente', 'meilleures', 'personnes', 'bois', 'poste', 'negociateur', 'toutes']
['pixel', 'buds', 'google', 'pixel', 'buds', 'google', 'pixel', 'buds', 'review', 'wireless', 'earbuds', 'review', 'google', 'assistant', 'airpods', 'translate', 'translation', 'the', 'verge', 'verge', 'sean', 'o', 'kane', 'google', '']
['touch', 'built', 'upgrade', 'required', 'astronaut', 'copies', 'protect', 'option', 'mansions', 'echo', 'cafe', 'tighten', 'initial', 'jackasses', 'enter', 'buying', 'riveting', 'ncuz', 'playstation']
['amelialiana', 'amelia', 'liana', 'asos', 'haul', 'unboxing', 'try', 'on', 'asos', 'sale', 'black', 'friday', 'cyber', 'monday', 'haul', 'asos', 'unboxing', '', '', 'black', 'friday', '', '', 'cyber', 'monday', '', '', 'try', 'on', 'style', 'fashion', '']
['nmaroon', 'waited', 'npurchase', 'nblue', 'string', 'rack', 'immortality', 'n컨투어', 'ncover', 'nmake', '

['sights', 'anymore', 'yo', 'astronaut', 'appropriately', 'hopeful', 'npick', 'everywhere', 'companyfull', 'unpopular', 'write', 'hurts', 'forgotten', 'tracks', 'tabbes', 'key', 'sponsor', 'beyoncé', 'ngive']
['noel', 'gallagher', 'noel', 'gallagher', 'high', 'flying', 'birds', 'oasis', 'chasing', 'yesterday', 'who', 'built', 'the', 'moon', 'holy', 'mountain', 'fort', 'knox', 'she', 'taught', 'me', 'how', 'to', 'fly', 'it', 's', 'a', 'beautiful', 'world', 'manchester', 'alternative', 'indie', 'rock', 'guitar', 'bbc', '', '', 'music', 'radio', '', '', 'new', 'music', 'neol', 'galagher', 'wonderwall', '']
['npick', 'hoping', 'solo', 'snowman', 'alienated', 'tb', 'companyfull', 'omid', 'tracks', 'rawat', 'farm', 'bulgaria', 'astronaut', 'snowmansiavevomove', 'barbies', 'forth', 'realization', 'forgotten', 'quiet']
['', 'none', '']
['hosts', 'joined', 'oliver', 'sheldon', 'controversial', 'jessica', 'ruhle', 'irwin', 'kane', 'sheamus', 'reigns', 'former', 'dicemsnbc', 'cesaro', 'looklate',

In [99]:
tags = recommendTags(word2vec_model, input_words=us_ca_gb_df_test.loc[253, 'video_features'], 
                     number_of_tags=AVG_TAGS_PER_VIDEO, 
                     model_name = 'word2vec_model.w2v')


In [100]:
us_ca_gb_df_test.loc[253, 'tags']

'Graham Norton Graham Norton Show Official Entertainment Chat Show Graham Norton full episodes the graham Norton show full eps graham Norton full eps watch graham Norton online jason momoa hugh grant sarah millican kelly clarkson graham norton red chair game of thrones '

In [101]:
tags

[('showit', 0.975365161895752),
 ('siriusxm', 0.9519914388656616),
 ('quindent', 0.9478841423988342),
 ('format', 0.9390561580657959),
 ('doctors', 0.9379031658172607),
 ('outta', 0.9369737505912781),
 ('employer', 0.9345635175704956),
 ('theater', 0.933597981929779),
 ('peek', 0.9322747588157654),
 ('bee', 0.9283744692802429),
 ('yeri', 0.9267011880874634),
 ('haim', 0.9253869652748108),
 ('consecutive', 0.9232497811317444),
 ('conan', 0.9226843118667603),
 ('seasons', 0.9213836193084717),
 ('demand', 0.9209454655647278),
 ('gurlej', 0.9203862547874451),
 ('members', 0.9200960397720337),
 ('injuries', 0.9200307130813599)]

In [102]:
us_ca_gb_df_test.loc[1043,'video_features']

['battle',
 'universe',
 'nerf',
 'battle',
 'universe',
 'nerf',
 'challenge',
 'epic',
 'nerf',
 'battle',
 'nerf',
 'gun',
 'fight',
 'nerf',
 'fight',
 'team',
 'edge',
 'hi',
 'hi',
 'nerf',
 'hi',
 'studios',
 'matthias',
 'vlogs',
 'nerf',
 'nurf',
 'nerf',
 'war',
 'nerf',
 'battle',
 'nerf',
 'dungeons',
 'dragons',
 'team',
 'edge',
 'nerf',
 'nerf',
 'war',
 'nerf',
 'attack',
 'nerf',
 'dungeons',
 'dragons',
 'challenge',
 'ep',
 'battle',
 'universe',
 'hope',
 'enjoyed',
 'newest',
 'episode',
 'nerf',
 'dungeons',
 'dragons',
 'challenge',
 'nerf',
 'challenges',
 'next',
 'answer',
 'comments',
 'n',
 'nthe',
 'official',
 'dungeons',
 'dragons',
 'challenge',
 'playlist',
 'universe',
 'twitter',
 'universe',
 'instagram',
 'family',
 'friendly',
 'channels',
 'hi',
 'studios',
 'n',
 'nmatthias',
 'vlogs',
 'amanda',
 'universe',
 'good',
 'edge',
 'gaming',
 'edge']

In [103]:
us_ca_gb_df_test

,index,video_id,trending_date,video_name,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,category,video_features,predicted_tags
1043,252,SfFKLEOOh2A,17.17.11,NERF Dungeons & Dragons Challenge! [Ep. 4],Battle Universe!,24,2017-11-15T22:00:00.000Z,Battle Universe Nerf Battle Universe Nerf Chal...,533948,12136,169,3638,https://i.ytimg.com/vi/SfFKLEOOh2A/default.jpg,False,False,False,Hope you enjoyed the newest episode of the NER...,Entertainment,"[battle, universe, nerf, battle, universe, ner...","[sub, negative, opening, wife, shooter, fire, ..."
253,253,8NHA23f7LvU,17.14.11,Jason Momoa Wows Hugh Grant With Some Dothraki...,The Graham Norton Show,24,2017-11-10T19:06:23.000Z,Graham Norton Graham Norton Show Official Ente...,1496225,16116,236,605,https://i.ytimg.com/vi/8NHA23f7LvU/default.jpg,False,False,False,I think Sarah Millican was very excited for th...,Entertainment,"[graham, norton, graham, norton, show, officia...","[showit, bee, reilly, roba, showbbc, cleared, ..."
2595,52,exLJr1ScSKY,17.28.11,What even is this? | The Weekly #44,SprinkleofChatter,22,2017-11-26T15:51:57.000Z,Louise Pentland,141211,6580,126,919,https://i.ytimg.com/vi/exLJr1ScSKY/default.jpg,False,False,False,What a freakin' week! A lot of stuff happened ...,People & Blogs,"[louise, pentlandwhat, even, weekly, sprinkleo...","[inspiring, nwhat, obstacles, hypercar, action..."
2901,358,8vambqBQVwk,17.28.11,WORTH THE HYPE?? GIGI HADID X MAYBELLINE,Samantha Maria,26,2017-11-22T19:50:14.000Z,samantha maria sammi beautycrush gigi hadid wo...,114101,2961,208,381,https://i.ytimg.com/vi/8vambqBQVwk/default.jpg,False,False,False,I got some of the GIGI X MAYBELLINE makeup so ...,Howto & Style,"[samantha, maria, sammi, beautycrush, gigi, ha...","[required, touch, repair, message, anymore, sh..."
1047,256,TCkl1eGmxTM,17.24.11,Pirates of the Caribbean: Dead Genres Tell No ...,Lindsay Ellis,24,2017-11-22T17:45:41.000Z,yt cc on,118481,9548,121,1310,https://i.ytimg.com/vi/TCkl1eGmxTM/default.jpg,False,False,False,"yo-ho yo-ho, Eisner/King Lear analogy\n\npatre...",Entertainment,"[yt, cc, onpirates, caribbean, dead, genres, t...","[ｗｈｅｒｅ, naitor, ntristan, n主頻道, lada, dựa, 보훈단..."
714,714,Wen6VQS6NG4,17.28.11,Giant Mousetrap powered Car,TheBackyardScientist,28,2017-11-23T02:47:34.000Z,mousetrap car thebackyardscientist Mousetrap c...,945598,19821,1920,2115,https://i.ytimg.com/vi/Wen6VQS6NG4/default.jpg,False,False,False,This is just a mini-video to announce i've bee...,Science & Technology,"[mousetrap, car, thebackyardscientist, mousetr...","[changes, effectively, hypercar, bed, entire, ..."
510,510,e_7zHm7GsYc,17.19.11,Hunter Hayes - You Should Be Loved (Part One O...,Hunter Hayes,10,2017-11-13T15:01:18.000Z,Hunter Hayes you should be loved the shadowbox...,112866,3643,112,137,https://i.ytimg.com/vi/e_7zHm7GsYc/default.jpg,False,False,False,You Should Be Loved (feat. The Shadowboxers) A...,Music,"[hunter, hayes, loved, shadowboxers, pictures,...","[yo, key, involved, dscvryxng, sights, dftl, g..."
2548,5,A_Z3AokMwWI,17.28.11,MINECON Earth 2017 Livestream,TeamMojang,20,2017-11-18T19:12:51.000Z,Minecon Minecraft Microsoft Mojang,1196407,0,0,7932,https://i.ytimg.com/vi/A_Z3AokMwWI/default.jpg,False,True,False,Pre-show starts immediately\nMINECON Earth sho...,Gaming,"[minecon, minecraft, microsoft, mojang, mineco...","[beating, hillyard, singular, ladies, surpassi..."
2006,1215,oWDBskFP2BI,17.28.11,C9LoL | New Player Announcement,C9ggTV,20,2017-11-27T18:01:25.000Z,cloud cloud gg league of legends cs go lol le...,82496,2492,272,758,https://i.ytimg.com/vi/oWDBskFP2BI/default.jpg,False,False,False,More info: http://cloud9.gg/news/svenskeren-jo...,Gaming,"[cloud, cloud, gg, league, legends, cs, go, lo...","[首要任務便是保護老虎, nelaine, krystal, n我知道我不是最会化妆的, t..."
2770,227,DIU3xPdhCBI,17.24.11,Sam Smith surprises brides at their wedding! (...,BBC Music,10,2017-11-09T22:26:10.